In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [24]:
# Get train/test dfs

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [25]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [26]:
# Separate X,y from train dataset:
tot_cols = train_df.columns
y_train_all = train_df.loc[:,'Survived']

# I consider only these as useful cols:
X_cols = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']
# I don't consider Name (probably useless), Ticket/Fare (same cleaner info is in Pclass), 
# 'Cabin' (could be interesting - maybe cabin number is related to lacation on boat - 
# but almost 80% of entries are nans.
# I decide to keep 'Embarked' as it maight be realted to cabin location/vicinity with escape exits
X_train_all = train_df.loc[:,X_cols]
X_test_all = test_df.loc[:,X_cols]

# Set PassengerId as index:
X_train_all = X_train_all.set_index('PassengerId')
X_test_all = X_test_all.set_index('PassengerId')

# In order to explore model accuracy before submitting to Kaggle competition, 
# I want to test accuracy on different models. I get 
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, train_size=0.8, test_size=0.2)

In [27]:
# Which cols have nans?

print('X_train:')
for col in X_train.columns:
    print(col, X_train[col].isnull().sum())

print('\nX_test:')
for col in X_test.columns:
    print(col, X_test[col].isnull().sum())

X_train:
Pclass 0
Sex 0
Age 139
SibSp 0
Parch 0
Embarked 2

X_test:
Pclass 0
Sex 0
Age 38
SibSp 0
Parch 0
Embarked 0


In [35]:
# Age column has some nans:
# print('Nans in Age / total: ', X_train['Age'].isna().sum()/len(X_train['Age']))
# Around 20% entries. But I expect other columns (as Pclass) to be higly informative, 
# so I'd rather not throw away any line.
# I substitute nans in Age column them with the mean:
X_train.Age = X_train.Age.fillna(X_train.Age.mean())
X_test.Age = X_test.Age.fillna(X_test.Age.mean())

# Embarked columns has 2 Nan. I fill them with most common entries:
X_train.Embarked = X_train.Embarked.fillna(X_train.Embarked.mode().iloc[0])
X_test.Embarked = X_test.Embarked.fillna(X_test.Embarked.mode().iloc[0])

# Sex and Age are cathegorical data. Let's switch them to numerical
# Since they are simple and few, I choose the Ordinal Encoding:
object_cols = ['Sex', 'Embarked']
ordinal_encoder = OrdinalEncoder()
X_train.loc[:,object_cols] = ordinal_encoder.fit_transform(X_train.loc[:,object_cols])
X_test.loc[:,object_cols] = ordinal_encoder.transform(X_test.loc[:,object_cols])

In [7]:
X_test

,Pclass,Sex,Age,SibSp,Parch,Embarked
PassengerId,,,,,,
95,3,1.0,59.00000,0,0,2.0
526,3,1.0,40.50000,0,0,1.0
506,1,1.0,18.00000,1,0,0.0
672,1,1.0,31.00000,1,0,2.0
481,3,1.0,9.00000,5,2,2.0
...,...,...,...,...,...,...
414,2,1.0,30.16958,0,0,2.0
710,3,1.0,30.16958,1,1,0.0
388,2,0.0,36.00000,0,0,2.0


# Testing different models:

Let's try SVM:

In [8]:
# Try different values for C parameter (regularization)
C_vals = [0.01, 0.1, 1.0, 10., 100.]
for C_val in C_vals:
    svm = SVC(kernel = 'linear', C=C_val).fit(X_train, y_train)
    print('C = ', C_val, ' --> accuracy = ', svm.score(X_test, y_test))
    
# Looks like a default C=1 value is ok.

C =  0.01  --> accuracy =  0.8379888268156425
C =  0.1  --> accuracy =  0.8044692737430168
C =  1.0  --> accuracy =  0.8044692737430168
C =  10.0  --> accuracy =  0.8044692737430168
C =  100.0  --> accuracy =  0.8044692737430168


In [9]:
# Let's try kernelized SVM:

# Try different values for gamma parameter (rbf Kernel width)
gamma_vals = [0.001, 0.01, 1.0, 10.0]
for gamma_val in gamma_vals:
    clf = SVC(kernel = 'rbf', gamma=gamma_val).fit(X_train, y_train)
    print('gamma = {:.2f} -- > accuracy = {:.2f}'.format(gamma_val, clf.score(X_test, y_test)))

gamma = 0.00 -- > accuracy = 0.67
gamma = 0.01 -- > accuracy = 0.79
gamma = 1.00 -- > accuracy = 0.77
gamma = 10.00 -- > accuracy = 0.68


In [10]:
# Let's tune C and gamma at the same time:
C_vals = [0.01, 0.1, 1.0, 10., 100.]
gamma_vals = [0.001, 0.01, 1.0, 10.0]
for C_val in C_vals:
    for gamma_val in gamma_vals:
        clf = SVC(kernel = 'rbf', C=C_val, gamma=gamma_val).fit(X_train, y_train)
        print('C = {:.2f}  gamma = {:.2f} -- > accuracy = {:.4f}'.format(C_val, gamma_val, clf.score(X_test, y_test)))
        
#Best combination seems: C = 10.00  gamma = 0.01 -- > accuracy = 0.8156

C = 0.01  gamma = 0.00 -- > accuracy = 0.6369
C = 0.01  gamma = 0.01 -- > accuracy = 0.6369
C = 0.01  gamma = 1.00 -- > accuracy = 0.6369
C = 0.01  gamma = 10.00 -- > accuracy = 0.6369
C = 0.10  gamma = 0.00 -- > accuracy = 0.6369
C = 0.10  gamma = 0.01 -- > accuracy = 0.6704
C = 0.10  gamma = 1.00 -- > accuracy = 0.6369
C = 0.10  gamma = 10.00 -- > accuracy = 0.6369
C = 1.00  gamma = 0.00 -- > accuracy = 0.6704
C = 1.00  gamma = 0.01 -- > accuracy = 0.7877
C = 1.00  gamma = 1.00 -- > accuracy = 0.7709
C = 1.00  gamma = 10.00 -- > accuracy = 0.6816
C = 10.00  gamma = 0.00 -- > accuracy = 0.8212
C = 10.00  gamma = 0.01 -- > accuracy = 0.8324
C = 10.00  gamma = 1.00 -- > accuracy = 0.7933
C = 10.00  gamma = 10.00 -- > accuracy = 0.6872
C = 100.00  gamma = 0.00 -- > accuracy = 0.8324
C = 100.00  gamma = 0.01 -- > accuracy = 0.8324
C = 100.00  gamma = 1.00 -- > accuracy = 0.7933
C = 100.00  gamma = 10.00 -- > accuracy = 0.6872


In [11]:
#Same as above cell but applying also MinMax scaler:

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Let's tune C and gamma at the same time:
C_vals = [0.01, 0.1, 1.0, 10., 100.]
gamma_vals = [0.001, 0.01, 1.0, 10.0]
for C_val in C_vals:
    for gamma_val in gamma_vals:
        clf = SVC(kernel = 'rbf', C=C_val, gamma=gamma_val).fit(X_train_scaled, y_train)
        print('C = {:.2f}  gamma = {:.2f} -- > accuracy = {:.4f}'.format(C_val, gamma_val, clf.score(X_test_scaled, y_test)))
        
#You can notice a slight overall improvement, but not on the highest achievable accuracy
# Now best combination seems C = 10.00  gamma = 1.00 -- > accuracy = 0.8156
# Different gamma param but same accuracy

C = 0.01  gamma = 0.00 -- > accuracy = 0.6369
C = 0.01  gamma = 0.01 -- > accuracy = 0.6369
C = 0.01  gamma = 1.00 -- > accuracy = 0.7151
C = 0.01  gamma = 10.00 -- > accuracy = 0.6369
C = 0.10  gamma = 0.00 -- > accuracy = 0.6369
C = 0.10  gamma = 0.01 -- > accuracy = 0.6369
C = 0.10  gamma = 1.00 -- > accuracy = 0.8156
C = 0.10  gamma = 10.00 -- > accuracy = 0.8436
C = 1.00  gamma = 0.00 -- > accuracy = 0.6369
C = 1.00  gamma = 0.01 -- > accuracy = 0.8045
C = 1.00  gamma = 1.00 -- > accuracy = 0.8436
C = 1.00  gamma = 10.00 -- > accuracy = 0.8436
C = 10.00  gamma = 0.00 -- > accuracy = 0.8045
C = 10.00  gamma = 0.01 -- > accuracy = 0.8045
C = 10.00  gamma = 1.00 -- > accuracy = 0.8547
C = 10.00  gamma = 10.00 -- > accuracy = 0.8156
C = 100.00  gamma = 0.00 -- > accuracy = 0.8045
C = 100.00  gamma = 0.01 -- > accuracy = 0.8045
C = 100.00  gamma = 1.00 -- > accuracy = 0.8492
C = 100.00  gamma = 10.00 -- > accuracy = 0.8045


In [12]:
# As above but adding cross-validation (to get more reliable predictions, since the dataset is small):

# Without variables rescaling:
C_vals = [0.01, 0.1, 1.0, 10., 100.]
gamma_vals = [0.001, 0.01, 1.0, 10.0]
print('WITHOUT VARIABLES RESCALING:\n')
for C_val in C_vals:
    for gamma_val in gamma_vals:
        clf = SVC(kernel = 'rbf', C=C_val, gamma=gamma_val)
        cv_scores = cross_val_score(clf, X_train, y_train)
        print('C = {:.2f}  gamma = {:.2f}:'.format(C_val, gamma_val))
        print('Cross-validation scores (3-fold):', cv_scores)
        print('Mean cross-validation score (3-fold): {:.3f}'.format(np.mean(cv_scores)))
        
        
# With variables rescaling:
C_vals = [0.01, 0.1, 1.0, 10., 100.]
gamma_vals = [0.001, 0.01, 1.0, 10.0]
print('\n\nWITH VARIABLES RESCALING:\n')
for C_val in C_vals:
    for gamma_val in gamma_vals:
        clf = SVC(kernel = 'rbf', C=C_val, gamma=gamma_val)
        cv_scores = cross_val_score(clf, X_train_scaled, y_train)
        print('C = {:.2f}  gamma = {:.2f}:'.format(C_val, gamma_val))
        print('Cross-validation scores (3-fold):', cv_scores)
        print('Mean cross-validation score (3-fold): {:.3f}'.format(np.mean(cv_scores)))

WITHOUT VARIABLES RESCALING:

C = 0.01  gamma = 0.00:
Cross-validation scores (3-fold): [0.60839161 0.60839161 0.61267606 0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.611
C = 0.01  gamma = 0.01:
Cross-validation scores (3-fold): [0.60839161 0.60839161 0.61267606 0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.611
C = 0.01  gamma = 1.00:
Cross-validation scores (3-fold): [0.60839161 0.60839161 0.61267606 0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.611
C = 0.01  gamma = 10.00:
Cross-validation scores (3-fold): [0.60839161 0.60839161 0.61267606 0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.611
C = 0.10  gamma = 0.00:
Cross-validation scores (3-fold): [0.60839161 0.60839161 0.61267606 0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.611
C = 0.10  gamma = 0.01:
Cross-validation scores (3-fold): [0.61538462 0.60839161 0.5915493  0.61267606 0.61267606]
Mean cross-validation score (3-fold): 0.608
C = 0.10  gamma

Let's try random forests:

In [13]:
# Simple example:

clf = RandomForestClassifier().fit(X_train, y_train)
clf.score(X_test, y_test)

0.8435754189944135

Let's try gradient boosting:

In [37]:
clf = XGBClassifier().fit(X_train, y_train)
print('Accuracy of XGBClassifier with default parameters: ', clf.score(X_test, y_test))

n_est_vals = [350,500,750]
learning_rate_vals = [0.01, 0.05, 0.1, 0.5, 1.0]

print('\nAccuracy of XGBClassifier with specific parameters values: ')
for n_est_val in n_est_vals:
    for learning_rate_val in learning_rate_vals:
        clf = XGBClassifier(n_estimators=n_est_val, learning_rate = learning_rate_val, eval_metric='logloss')\
        .fit(X_train, y_train, verbose=False)
        print('n_estimators = {:.2f}  learning_rate = {:.2f} --> accuracy = {:.3f}'\
              .format(n_est_val, learning_rate_val ,clf.score(X_test, y_test)))
        

[10:42:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of XGBClassifier with default parameters:  0.7877094972067039

Accuracy of XGBClassifier with specific parameters values: 
n_estimators = 350.00  learning_rate = 0.01 --> accuracy = 0.810
n_estimators = 350.00  learning_rate = 0.05 --> accuracy = 0.804
n_estimators = 350.00  learning_rate = 0.10 --> accuracy = 0.804
n_estimators = 350.00  learning_rate = 0.50 --> accuracy = 0.782
n_estimators = 350.00  learning_rate = 1.00 --> accuracy = 0.788
n_estimators = 500.00  learning_rate = 0.01 --> accuracy = 0.816
n_estimators = 500.00  learning_rate = 0.05 --> accuracy = 0.799
n_estimators = 500.00  learning_rate = 0.10 --> accuracy = 0.804
n_estimators = 500.00  learning_rate = 0.50 --> accuracy = 0.782
n_estimators = 500.00  learning

In [38]:
# Same as above but with cross-validation:

for n_est_val in n_est_vals:
    for learning_rate_val in learning_rate_vals:
        clf = XGBClassifier(n_estimators=n_est_val, learning_rate = learning_rate_val, eval_metric='logloss')
        cv_scores = cross_val_score(clf, X_train, y_train)
        print('n_estimators = {:.2f}  learning_rate = {:.2f} --> '\
              .format(n_est_val, learning_rate_val))
        print('Cross-validation scores (3-fold):', cv_scores)
        print('Mean cross-validation score (3-fold): {:.3f}'.format(np.mean(cv_scores)))
        
        
# Best one appears to be n_estimators = 500.00  learning_rate = 0.01 --> 
# Cross-validation scores (3-fold): [0.83216783 0.85314685 0.82394366 0.81690141 0.81690141]
# Mean cross-validation score (3-fold): 0.829

n_estimators = 350.00  learning_rate = 0.01 --> 
Cross-validation scores (3-fold): [0.82517483 0.81118881 0.86619718 0.78169014 0.75352113]
Mean cross-validation score (3-fold): 0.808
n_estimators = 350.00  learning_rate = 0.05 --> 
Cross-validation scores (3-fold): [0.81818182 0.79020979 0.84507042 0.77464789 0.76056338]
Mean cross-validation score (3-fold): 0.798
n_estimators = 350.00  learning_rate = 0.10 --> 
Cross-validation scores (3-fold): [0.83916084 0.78321678 0.82394366 0.77464789 0.74647887]
Mean cross-validation score (3-fold): 0.793
n_estimators = 350.00  learning_rate = 0.50 --> 
Cross-validation scores (3-fold): [0.81818182 0.79020979 0.80985915 0.78169014 0.74647887]
Mean cross-validation score (3-fold): 0.789
n_estimators = 350.00  learning_rate = 1.00 --> 
Cross-validation scores (3-fold): [0.7972028  0.78321678 0.82394366 0.78873239 0.75352113]
Mean cross-validation score (3-fold): 0.789
n_estimators = 500.00  learning_rate = 0.01 --> 
Cross-validation scores (3-fold

In [39]:
# Same as above but with cross-validation + rescaled variables:
, eval_metric='logloss'
for n_est_val in n_est_vals:
    for learning_rate_val in learning_rate_vals:
        clf = XGBClassifier(n_estimators=n_est_val, learning_rate = learning_rate_val, eval_metric='logloss')
        cv_scores = cross_val_score(clf, X_train_scaled, y_train)
        print('n_estimators = {:.2f}  learning_rate = {:.2f} --> '\
              .format(n_est_val, learning_rate_val))
        print('Cross-validation scores (3-fold):', cv_scores)
        print('Mean cross-validation score (3-fold): {:.3f}'.format(np.mean(cv_scores)))
        
# Basically no difference compared to before

n_estimators = 350.00  learning_rate = 0.01 --> 
Cross-validation scores (3-fold): [0.58741259 0.58041958 0.58450704 0.56338028 0.58450704]
Mean cross-validation score (3-fold): 0.580
n_estimators = 350.00  learning_rate = 0.05 --> 
Cross-validation scores (3-fold): [0.56643357 0.57342657 0.61267606 0.57746479 0.5915493 ]
Mean cross-validation score (3-fold): 0.584
n_estimators = 350.00  learning_rate = 0.10 --> 
Cross-validation scores (3-fold): [0.60839161 0.59440559 0.57746479 0.54225352 0.55633803]
Mean cross-validation score (3-fold): 0.576
n_estimators = 350.00  learning_rate = 0.50 --> 
Cross-validation scores (3-fold): [0.55244755 0.57342657 0.54929577 0.54225352 0.59859155]
Mean cross-validation score (3-fold): 0.563
n_estimators = 350.00  learning_rate = 1.00 --> 
Cross-validation scores (3-fold): [0.56643357 0.58741259 0.51408451 0.57746479 0.59859155]
Mean cross-validation score (3-fold): 0.569
n_estimators = 500.00  learning_rate = 0.01 --> 
Cross-validation scores (3-fold

# Final model and prediction:
(choosing best model from above tests)

In [40]:
# Train model over the full original training dataset

train_df = train_df.loc[:,X_cols]
test_df = test_df.loc[:,X_cols]
#Set PassengerId as index:
X_train = X_train_all
X_test = X_test_all
y_train = y_train_all

# Remove Nan:
X_train.Age = X_train.Age.fillna(X_train.Age.mean())
X_test.Age = X_test.Age.fillna(X_test.Age.mean())
# Embarked columns has 2 Nan. I fill them with most common entries:
X_train.Embarked = X_train.Embarked.fillna(X_train.Embarked.mode()[0])
X_test.Embarked = X_test.Embarked.fillna(X_test.Embarked.mode()[0])

# Sex and Age are cathegorical data. Let's switch them to numerical
object_cols = ['Sex', 'Embarked']
ordinal_encoder = OrdinalEncoder()
X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
X_test[object_cols] = ordinal_encoder.transform(X_test[object_cols])

# Chosen model is GradientBoosting with n_estimators = 500.00  learning_rate = 0.01
# (but also SVM with tuned params would give similar accuracy on test data)
clf = XGBClassifier(n_estimators=500, learning_rate = 0.01, eval_metric='logloss')
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [18]:
output = pd.DataFrame({'PassengerId': X_test.index,
                       'Survived': predictions})
output.to_csv('submission.csv', index=False)

This predicted output scored an accuracy of almost 80% after submission to Kaggle competition.